This is intended to be a simple notebook for going through analysis of the Perturb-CITE-seq data, and get a better sense of what the data looks like. 

Ressources:
1. [Paper](https://www.nature.com/articles/s41588-021-00779-1)
2. [Original code for reproducing the paper](https://github.com/klarman-cell-observatory/Perturb-CITE-seq) 
3. [Data release](https://singlecell.broadinstitute.org/single_cell/study/SCP1064/multi-modal-pooled-perturb-cite-seq-screens-in-patient-models-define-novel-mechanisms-of-cancer-immune-evasion)

# Step 0a: Download the data onto the Colab hardware

This (unfortunately) requires you to go through the following steps:
1. Go to the data release [page](https://singlecell.broadinstitute.org/single_cell/study/SCP1064/multi-modal-pooled-perturb-cite-seq-screens-in-patient-models-define-novel-mechanisms-of-cancer-immune-evasion)
2. Login / Create an account
3. Click on Download
4. Click on Bulk Download option
5. Copy **only** the variable *auth_code* from the URL and paste it **onto** the auth_code of the URL for the curl command below (each authorization code only works for 30 minute), and run the command (had to customize it).

It should take under two minute

In [1]:
!curl -k "https://singlecell.broadinstitute.org/single_cell/api/v1/bulk_download/generate_curl_config?accessions=SCP1064&auth_code=Qo1e5U9N&directory=all" -o cfg.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  7154    0  7154    0     0   3753      0 --:--:--  0:00:01 --:--:--  3753


In [2]:
!curl -k -K cfg.txt && rm cfg.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  833M  100  833M    0     0  30.7M      0  0:00:27  0:00:27 --:--:-- 31.0M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1341k  100 1341k    0     0  5543k      0 --:--:-- --:--:-- --:--:-- 5566k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1879k  100 1879k    0     0  5177k      0 --:--:-- --:--:-- --:--:-- 5163k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  895M  100  895M    0     0  31.3M      0  0:00:28  0:00:28 --:--:-- 32.0M
  % Total    % Received % Xferd  Average Speed   Tim

# Step 1: Load all the data onto scanpy

In [1]:
import scanpy as sc
import pandas as pd
import numpy as np

In [2]:
# make sparse and dump
import scipy.sparse as sp

In [3]:
data = sc.read_csv("SCP1064/other/RNA_expression.csv.gz").transpose()

In [4]:
# sparsify!
data_sp = sp.csr_matrix(data.X)

In [5]:
# get covariates 
covariates =  pd.read_csv("SCP1064/metadata/RNA_metadata.csv", index_col=0).iloc[1:, ]
data.obs = covariates

/home/ubuntu/anaconda3/envs/pytorch/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (3,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
# correct dtypes
data.obs["MOI"] = data.obs["MOI"].astype(np.int32)
data.obs["UMI_count"] = data.obs["UMI_count"].astype(np.double)

In [7]:
# de-normalize and round up
norm_factor =  data.obs["UMI_count"].values / 1.0e6
Z = sp.diags(norm_factor).dot(np.expm1(data_sp))
print(np.greater_equal(np.abs(Z.data - np.rint(Z.data)), 0.01).any())
Z.data = np.rint(Z.data)
data.X = Z

False


In [8]:
# read guide info
guide_info = pd.read_csv("SCP1064/documentation/all_sgRNA_assignments.txt", index_col=0)
guide_info = guide_info.replace(np.nan,'',regex=True)
data.obs["sgRNAs"] = guide_info["sgRNAs"].astype(str)

In [10]:
data.write_h5ad("SCP1064/other/adata.h5ad")

/home/ubuntu/anaconda3/envs/pytorch/lib/python3.9/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'library_preparation_protocol' as categorical
/home/ubuntu/anaconda3/envs/pytorch/lib/python3.9/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'condition' as categorical
/home/ubuntu/anaconda3/envs/pytorch/lib/python3.9/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is depreca

In [9]:
guide_info

,sgRNAs
Cell,
CELL_1,HLA-B_2
CELL_2,"NGFR_3,SERPINF1_3"
CELL_3,HLA-B_2
CELL_4,"NMRK1_3,S100A6_3"
CELL_5,
...,...
CELL_218327,"CTSO_3,PIK3IP1_3,VDAC2_2,WNT7A_1"
CELL_218328,"JAK2_3,SLC7A5P1_3"
CELL_218329,"S100A6_2,SAT1_2,ONE_NON-GENE_SITE_177"


In [ ]:
covariates["condition"].value_counts()

IFNγ          87590
Co-culture    73114
Control       57627
Name: condition, dtype: int64

In [ ]:
# Multiplicity of infection = number of guides per cell?
# Extended figure 2
covariates["MOI"].value_counts()

1     126966
2      45135
0      23028
3      14525
4       5053
5       1942
6        793
7        386
8        222
9        115
10        64
11        37
12        30
13        11
15         9
14         6
17         4
18         2
19         2
16         1
Name: MOI, dtype: int64

In [ ]:
covariates["MOI"].mean()

1.3874850570922133

In [ ]:
# maybe the first occurence only? otherwise there should be many more combinations: YES, full assignment is below
covariates["sgRNA"].value_counts()

IFNGR2_2       358
NO_SITE_47     333
NO_SITE_913    317
HLA-DRB5_2     315
NO_SITE_36     296
              ... 
DNAJC9_2         2
ATP1A1_1         2
EIF2S3_3         1
UBC_2            1
TUBB_2           1
Name: sgRNA, Length: 818, dtype: int64